COMP90024 Project Team 1

Authors:
- Henrik Hao (1255309)
- Haoyi Li (1237964)
- Zilin Su (1155122)
- Angela Yifei Yuan (1269549)

In [2]:
pip install elasticsearch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 477.5/477.5 kB 594.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 417.5 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [50]:
import geopandas as gpd
import pandas as pd
import json
from shapely.geometry import mapping

In [8]:
from elasticsearch import Elasticsearch
import warnings
warnings.filterwarnings("ignore")

es=Elasticsearch("https://127.0.0.1:9200", http_auth=('elastic','elastic'), verify_certs=False)
index_list = es.cat.indices(format='json')
print(index_list)


[{'health': 'green', 'status': 'open', 'index': 'pha_sa2', 'uuid': 'uObZU8j4Qf-DwUBWK4JOfA', 'pri': '1', 'rep': '1', 'docs.count': '2383', 'docs.deleted': '0', 'store.size': '340.9kb', 'pri.store.size': '165.5kb'}, {'health': 'green', 'status': 'open', 'index': 'hospital_admissions', 'uuid': 'VsWHbbk9Rsm2b8WAyvHF3A', 'pri': '1', 'rep': '1', 'docs.count': '3989', 'docs.deleted': '0', 'store.size': '671kb', 'pri.store.size': '335kb'}, {'health': 'green', 'status': 'open', 'index': 'observations', 'uuid': 'n8SzGSC1SIy4wReSi2oF4A', 'pri': '1', 'rep': '1', 'docs.count': '0', 'docs.deleted': '0', 'store.size': '450b', 'pri.store.size': '225b'}, {'health': 'green', 'status': 'open', 'index': 'pha', 'uuid': 'sVeVXZpyRECso9iUTpbseA', 'pri': '1', 'rep': '1', 'docs.count': '1165', 'docs.deleted': '0', 'store.size': '103.9kb', 'pri.store.size': '49.7kb'}]


In [133]:
# create index
index_name = 'bushfire'
index_settings = {
    "settings": {
        "index": {
            "number_of_shards": 3,
            "number_of_replicas": 1
        }
    },
    "mappings": {
        "properties": {
            "ignition_date": {
                "type": "keyword"
            },
            "fire_type": {
                "type": "text"
            },
            "area_ha": {
                "type": "integer"
            },
            "state": {
                "type": "text"
            },
            "geometry": {
                "type": "geo_shape"
            },
            
        }
    }
}

es.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'bushfire'})

In [131]:
#es.indices.delete(index=index_name, ignore=[400,404])
# 跑python前删了

ObjectApiResponse({'acknowledged': True})

In [70]:
# insert documents
original_bushfire = gpd.read_file('dataset/bushfire_files/Historical_Bushfire_Boundaries.geojson')



In [150]:
# prepcocessing bushfire data
bushfire = original_bushfire[['ignition_date', 'fire_type', 'area_ha', 'state', 'geometry']]
bushfire = bushfire[(bushfire['ignition_date'].dt.year >= 2013) & (bushfire['ignition_date'].dt.year <= 2021)]
#bushfire = bushfire[(bushfire['ignition_date'].dt.year >= 2013)]
# transform timestamp to time string
bushfire['ignition_date'] = bushfire['ignition_date'].dt.strftime('%Y-%m-%d')
bushfire = bushfire.dropna()




In [152]:
bushfire = bushfire[bushfire['fire_type']=='Bushfire']
bushfire = bushfire[bushfire['state'] == 'NSW (New South Wales)']
bushfire


,ignition_date,fire_type,area_ha,state,geometry
81,2017-08-18,Bushfire,883,NSW (New South Wales),"MULTIPOLYGON (((152.80331 -29.15075, 152.80361..."
188,2017-02-04,Bushfire,465,NSW (New South Wales),"POLYGON ((152.12168 -28.91521, 152.12162 -28.9..."
202,2014-01-21,Bushfire,3,NSW (New South Wales),"POLYGON ((152.07697 -28.50465, 152.07714 -28.5..."
368,2017-08-16,Bushfire,3508,NSW (New South Wales),"POLYGON ((152.41199 -31.07210, 152.41217 -31.0..."
843,2018-09-19,Bushfire,0,NSW (New South Wales),"POLYGON ((151.20999 -33.63465, 151.21044 -33.6..."
...,...,...,...,...,...
309991,2017-10-29,Bushfire,128,NSW (New South Wales),"POLYGON ((151.32903 -32.07808, 151.32783 -32.0..."
310036,2020-08-21,Bushfire,33,NSW (New South Wales),"POLYGON ((153.45141 -28.37879, 153.45139 -28.3..."
310124,2014-01-18,Bushfire,274,NSW (New South Wales),"POLYGON ((147.76441 -35.94698, 147.76444 -35.9..."
310155,2017-10-07,Bushfire,3,NSW (New South Wales),"POLYGON ((151.48478 -33.39011, 151.48475 -33.3..."


In [185]:
bushfire.to_file("dataset/bushfire_files/filtered_bushfire.geojson", driver='GeoJSON')

## store into Elastic Search


In [153]:
for _, row in bushfire.iterrows():
    record = row.to_dict()
    record['geometry'] = json.loads(json.dumps(mapping(record['geometry'])))
    es.index(index=index_name, body=record)



## Try some quries

In [154]:
## check
query = {'query': {'match_all':{}}}
result = es.search(index=index_name, body=query,size=10)
print(result)

{'took': 5304, 'timed_out': False, '_shards': {'total': 3, 'successful': 3, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 3277, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'bushfire', '_id': 'XMqEQ48BSwiTck_yvjWg', '_score': 1.0, '_source': {'ignition_date': '2014-01-21', 'fire_type': 'Bushfire', 'area_ha': 3, 'state': 'NSW (New South Wales)', 'geometry': {'type': 'Polygon', 'coordinates': [[[152.076970679423, -28.5046481798538], [152.077142092382, -28.5048195908716], [152.077656326335, -28.5047338849207], [152.077742032314, -28.5048195909296], [152.078170561213, -28.5052481219744], [152.078170561096, -28.5061908859817], [152.077056385158, -28.506705121878], [152.076199326319, -28.5061908857911], [152.076199326436, -28.5052481217838], [152.076970679423, -28.5046481798538]]]}}}, {'_index': 'bushfire', '_id': 'XcqEQ48BSwiTck_yvzWX', '_score': 1.0, '_source': {'ignition_date': '2017-08-16', 'fire_type': 'Bushfire', 'area_ha': 3508, 'state': 'NSW (New South Wales)', 

In [184]:
query = {
      "query": {
        "match": {
          "area_ha": 878
        }
      }
    }

result = es.search(index=index_name, body=query)
for hit in result['hits']['hits']:
    print(hit['_source'])

{'ignition_date': '2015-10-06', 'fire_type': 'Bushfire', 'area_ha': 878, 'state': 'NSW (New South Wales)', 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[152.556150367635, -29.7137939029147], [152.556235526758, -29.7127700509153], [152.556310155815, -29.7122558879188], [152.556435840848, -29.7118862929283], [152.556084139884, -29.711930238894], [152.556037017889, -29.7119344868894], [152.556091220892, -29.7118625768942], [152.556145459918, -29.7116121958976], [152.556146228918, -29.7116086458977], [152.55614885192, -29.7115965388979], [152.556168387929, -29.7115063568991], [152.556000617975, -29.711307248881], [152.555926975984, -29.7113024898738], [152.555850660998, -29.711269930866], [152.555745749016, -29.7112251718553], [152.555498155051, -29.7111787098305], [152.555427460061, -29.7111654438235], [152.554994423157, -29.7108255607782], [152.554685420208, -29.7107117137468], [152.554512014232, -29.7106930807296], [152.554506307233, -29.710688549729], [152.554466619242, -29.7

In [160]:
len(result['hits']['hits'])

10

In [178]:
# json
result['hits']['hits'][9]


{'_index': 'bushfire',
 '_id': 'dsqEQ48BSwiTck_y8DUI',
 '_score': 1.0,
 '_source': {'ignition_date': '2015-10-06',
  'fire_type': 'Bushfire',
  'area_ha': 878,
  'state': 'NSW (New South Wales)',
  'geometry': {'type': 'MultiPolygon',
   'coordinates': [[[[152.556150367635, -29.7137939029147],
      [152.556235526758, -29.7127700509153],
      [152.556310155815, -29.7122558879188],
      [152.556435840848, -29.7118862929283],
      [152.556084139884, -29.711930238894],
      [152.556037017889, -29.7119344868894],
      [152.556091220892, -29.7118625768942],
      [152.556145459918, -29.7116121958976],
      [152.556146228918, -29.7116086458977],
      [152.55614885192, -29.7115965388979],
      [152.556168387929, -29.7115063568991],
      [152.556000617975, -29.711307248881],
      [152.555926975984, -29.7113024898738],
      [152.555850660998, -29.711269930866],
      [152.555745749016, -29.7112251718553],
      [152.555498155051, -29.7111787098305],
      [152.555427460061, -29.71116

In [181]:
result['hits']['hits'][9].keys()

dict_keys(['_index', '_id', '_score', '_source'])